In [ ]:
import numpy as np
from scipy.stats import poisson_binom
from scipy.stats import binom
from functools import cache
from collections.abc import Sequence
import pandas as pd
from dataclasses import dataclass, astuple
from itertools import product

In [ ]:
trad_system = PbftParameters(4,3,4,3,2)
wtf = PbftParameters(5,3,3,3,2)
new_system = PbftParameters(5,4,4,4,2)
big_trad = PbftParameters(7,5,5,5,3)
new_big = PbftParameters(8,6,6,6,3)
biggest_trad = PbftParameters(10,7,7,7,4)
print(pbft_min_unsafe_failures(trad_system), pbft_min_unlive_failures(trad_system))
print(pbft_min_unsafe_failures(wtf), pbft_min_unlive_failures(wtf))
print(pbft_min_unsafe_failures(new_system), pbft_min_unlive_failures(new_system))
print(pbft_min_unsafe_failures(new_system), pbft_min_unlive_failures(big_trad))
print(pbft_min_unsafe_failures(new_big), pbft_min_unlive_failures(new_big))
print(pbft_min_unsafe_failures(biggest_trad), pbft_min_unlive_failures(biggest_trad))

print(p_pbft_safe(new_big, 0.01))

In [ ]:
@dataclass
class PbftParameters:
    num_nodes: int
    prepare_qs: int
    commit_qs: int
    view_change_qs: int
    view_sync_qs: int


@dataclass
class RaftParameters:
    num_nodes: int
    commit_qs: int
    leader_election_qs: int

In [ ]:
raft_min_unlive_failures(RaftParameters(6,4,4))

In [ ]:
p = [0.08]*(7-4) + [.01]*4
rv = poisson_binom(p)
print(rv.cdf(3))

In [ ]:
old_val = 1
for i in range(0,6):
    p = [0.08]*(5-i) + [.01]*i
    rv = poisson_binom(p)
    cur_val = rv.cdf(2)
    print(f'i={i}: {cur_val*100} \t delta={(1-.9954747392000002)/(1-cur_val)}')
    old_val = cur_val

In [ ]:

for i in range(0,8):
    p = [0.08]*(7-i) + [.01]*i
    rv = poisson_binom(p)
    cur_val = rv.cdf(3)
    print(f'i={i}: {cur_val} \t delta={(1-.9988237205504003)/(1-cur_val)}')

In [ ]:
p_poisson_raft_safe_and_live(RaftParameters(5,3,3), [0.01]*3)

In [ ]:
from math import comb
from decimal import Decimal, getcontext

getcontext().prec = 100  # Set desired precision

@cache
def decimal_binom_range(num_trials: int, p_success: Decimal, *, min_success=None, max_success=None):
    total_probability = Decimal(0.0)

    for x in range(min_success, max_success + 1):
        # Calculate the probability for exactly x successes
        prob_x = comb(num_trials, x) * (p_success ** x) * ((1 - p_success) ** (num_trials - x))
        total_probability += prob_x

    return total_probability

def p_decimal_raft_safe_and_live(raft_parameters: RaftParameters, p_failure: Decimal) -> float:
    min_unlive_failures = raft_min_unlive_failures(raft_parameters)
    max_unlive_failures = raft_max_unlive_failures(raft_parameters)
    min_unsafe_failures = raft_min_unsafe_failures(raft_parameters)
    max_unsafe_failures = raft_max_unsafe_failures(raft_parameters)

    min_unsafe_or_unlive_failures = min(min_unlive_failures, min_unsafe_failures)
    max_unsafe_or_unlive_failures = max(max_unlive_failures, max_unsafe_failures)

    return 1 - decimal_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unsafe_or_unlive_failures,
        max_success=max_unsafe_or_unlive_failures,
    )

p_decimal_raft_safe_and_live(RaftParameters(11,6,6), Decimal(.01))

In [ ]:
@cache
def p_binom_range(
    num_trials: int, p_success: float, *, min_success=None, max_success=None
) -> float:
    if min_success is None:
        min_success = 0
    if max_success is None:
        max_success = num_trials
    if min_success > max_success:
        return 0
    return binom.cdf(max_success, num_trials, p_success) - binom.cdf(
        min_success - 1, num_trials, p_success
    )


def p_poisson_binom_range(num_trials: int, p_successes: list[float], * , min_success=None, max_success=None) -> float:
    if min_success is None:
        min_success = 0
    if max_success is None:
        max_success = num_trials
    if min_success > max_success:
        return 0

    ub = None
    lb = None
    if max_success >= num_trials:
        ub = 1
    else:
        ub = poisson_binom.cdf(max_success, num_trials, p_successes)
    if min_success <= 0:
        lb = 0
    else:
        lb = poisson_binom.cdf(
        min_success - 1, num_trials, p_successes
    )
    return ub - lb

def pbft_min_unsafe_failures(pbft_parameters: PbftParameters) -> int:
    """
    By Inclusion Exclusion: [|quorum1 U quorum2| <= N] === [|quorum1| + |quorum2| - |quorum1 \\intersect quorum2| <= N]
    The case where [|quorum1| + |quorum2| - Byz <= N] is bad as quorum1, quorum2 could have only byz intersection
    Writing it differently: [|quorum1| + |quorum2| - Byz <= N] === [|quorum1| + |quorum2| - N <= Byz]
    Therefore if [Byz >= |quorum1| + |quorum2| - N] then quorum1 and quorum2 may only have byz intersection
    """
    # Equivocation requires >= 1 Byz node and prepare quorums to intersect in only byz nodes
    min_failures_equivocate_prepare = max(
        pbft_parameters.prepare_qs
        + pbft_parameters.prepare_qs
        - pbft_parameters.num_nodes,
        1,
    )
    # Losing a commit requires commit and view change quorums to not intersect
    min_failures_non_persist_commit = (
        pbft_parameters.commit_qs
        + pbft_parameters.view_change_qs
        - pbft_parameters.num_nodes
    )

    return min(min_failures_equivocate_prepare, min_failures_non_persist_commit)


def pbft_max_unsafe_failures(pbft_parameters: PbftParameters) -> int:
    # There must be at least one non-byzantine node to create a safety violation
    return pbft_parameters.num_nodes


def pbft_min_unlive_failures(pbft_parameters: PbftParameters) -> int:
    # If [Correct <= biggest_quorum - 1] then we will never complete all quorums needed for consensus
    # Rewriting it gives: [N - Byz <= biggest_quorum - 1] === [Byz >= N - biggest_quorum + 1]
    biggest_quorum_size = max(
        pbft_parameters.prepare_qs,
        pbft_parameters.commit_qs,
        pbft_parameters.view_change_qs,
    )
    min_failures_to_not_progress = pbft_parameters.num_nodes - biggest_quorum_size + 1

    # PBFT provides that after 1 honest node enters the view, all honest nodes will enter in a bounded time post GST
    # PBFT gets this requirement because honest node needs view_change_qs messages, and view_change_qs-Byz will broadcast this
    # Then if any honest nodes hear view_sync_qs messages they will broadcast -- therefore we need [view_change_qs - Byz > view_sync_qs - 1]
    # If [view_change_qs - Byz <= view_sync_qs - 1] then we won't hit the view-sync_qs threshold
    # Equiv [Byz >= view_change_qs - view_sync_qs + 1]
    min_failures_to_not_sync_views = (
        pbft_parameters.view_change_qs - pbft_parameters.view_sync_qs + 1
    )

    # if this holds, byz nodes can force constant view changes
    min_failures_spam_view_change = min(
        pbft_parameters.view_sync_qs, pbft_parameters.view_change_qs
    )

    return min(
        min_failures_to_not_progress,
        min_failures_to_not_sync_views,
        min_failures_spam_view_change,
    )


def pbft_max_unlive_failures(pbft_parameters: PbftParameters) -> int:
    # Pbft can be unlive no matter how many nodes fail -- i.e. unlive when [min_unlive_failures <= Byz <= __N__]
    return pbft_parameters.num_nodes


def raft_min_unsafe_failures(raft_parameters: RaftParameters) -> int:
    """
    By Inclusion Exclusion: [|quorum1 U quorum2| <= N] === [|quorum1| + |quorum2| - |quorum1 \\intersect quorum2| <= N]
    The case where [|quorum1| + |quorum2| - 0 <= N] is bad as quorum1, quorum2 could have no intersection
    Raft requires any two leader election quorums to intersect AND the leader election and commit quorums to intersect
    Note that this is __not__ dependent on the number of nodes which fail
    """
    do_leader_and_commit_not_intersect = (
        raft_parameters.commit_qs + raft_parameters.leader_election_qs
        <= raft_parameters.num_nodes
    )
    do_leader_election_quorums_not_intersect = (
        raft_parameters.leader_election_qs + raft_parameters.leader_election_qs
        <= raft_parameters.num_nodes
    )
    # If there are <2 nodes there cannot be a safety violation
    is_safety_violation_possible = raft_parameters.num_nodes >= 2

    is_raft_unsafe = (
        do_leader_and_commit_not_intersect or do_leader_election_quorums_not_intersect
    ) and is_safety_violation_possible
    if is_raft_unsafe:
        # raft is unsafe even if no nodes fail
        return 0
    # raft is safe even if all nodes fail
    return raft_parameters.num_nodes + 1


def raft_max_unsafe_failures(raft_parameters: RaftParameters) -> int:
    # If there are <2 nodes there cannot be a safety violation
    is_violation_possible = raft_parameters.num_nodes >= 2
    if is_violation_possible:
        return raft_parameters.num_nodes
    # raft cannot be unsafe
    return -1


def raft_min_unlive_failures(raft_parameters: RaftParameters) -> int:
    # If [Correct <= biggest_quorum - 1] then we will never complete all quorums needed for consensus
    # Rewriting it gives: [N - Crashed <= biggest_quorum - 1] === [Crashed >= N - biggest_quorum + 1]
    biggest_quorum_size = max(
        raft_parameters.commit_qs, raft_parameters.leader_election_qs
    )
    min_failures_to_not_progress = raft_parameters.num_nodes - biggest_quorum_size + 1

    return min_failures_to_not_progress


def raft_max_unlive_failures(raft_parameters: RaftParameters) -> int:
    # raft can be unlive no matter how many nodes fail
    return raft_parameters.num_nodes


def p_pbft_safe(pbft_parameters: PbftParameters, p_failure: float) -> float:
    min_unsafe_failures = pbft_min_unsafe_failures(pbft_parameters)
    max_unsafe_failures = pbft_max_unsafe_failures(pbft_parameters)

    return 1 - p_binom_range(
        num_trials=pbft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unsafe_failures,
        max_success=max_unsafe_failures,
    )


def p_pbft_live(pbft_parameters: PbftParameters, p_failure: float) -> float:
    min_unlive_failures = pbft_min_unlive_failures(pbft_parameters)
    max_unlive_failures = pbft_max_unlive_failures(pbft_parameters)

    return 1 - p_binom_range(
        num_trials=pbft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unlive_failures,
        max_success=max_unlive_failures,
    )


def p_pbft_safe_and_live(pbft_parameters: PbftParameters, p_failure: float) -> float:
    min_unlive_failures = pbft_min_unlive_failures(pbft_parameters)
    max_unlive_failures = pbft_max_unlive_failures(pbft_parameters)
    min_unsafe_failures = pbft_min_unsafe_failures(pbft_parameters)
    max_unsafe_failures = pbft_max_unsafe_failures(pbft_parameters)

    min_unsafe_or_unlive_failures = min(min_unlive_failures, min_unsafe_failures)
    max_unsafe_or_unlive_failures = max(max_unlive_failures, max_unsafe_failures)

    return 1 - p_binom_range(
        num_trials=pbft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unsafe_or_unlive_failures,
        max_success=max_unsafe_or_unlive_failures,
    )


def p_raft_safe(raft_parameters: RaftParameters, p_failure: float) -> float:
    min_unsafe_failures = raft_min_unsafe_failures(raft_parameters)
    max_unsafe_failures = raft_max_unsafe_failures(raft_parameters)

    return 1 - p_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unsafe_failures,
        max_success=max_unsafe_failures,
    )


def p_raft_live(raft_parameters: RaftParameters, p_failure: float) -> float:
    min_unlive_failures = raft_min_unlive_failures(raft_parameters)
    max_unlive_failures = raft_max_unlive_failures(raft_parameters)

    return 1 - p_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unlive_failures,
        max_success=max_unlive_failures,
    )


def p_raft_safe_and_live(raft_parameters: RaftParameters, p_failure: float) -> float:
    min_unlive_failures = raft_min_unlive_failures(raft_parameters)
    max_unlive_failures = raft_max_unlive_failures(raft_parameters)
    min_unsafe_failures = raft_min_unsafe_failures(raft_parameters)
    max_unsafe_failures = raft_max_unsafe_failures(raft_parameters)

    min_unsafe_or_unlive_failures = min(min_unlive_failures, min_unsafe_failures)
    max_unsafe_or_unlive_failures = max(max_unlive_failures, max_unsafe_failures)

    return 1 - p_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_success=p_failure,
        min_success=min_unsafe_or_unlive_failures,
        max_success=max_unsafe_or_unlive_failures,
    )

############

def p_poisson_pbft_safe(pbft_parameters: PbftParameters, p_failures: list[float]) -> float:
    min_unsafe_failures = pbft_min_unsafe_failures(pbft_parameters)
    max_unsafe_failures = pbft_max_unsafe_failures(pbft_parameters)

    return 1 - p_poisson_binom_range(
        num_trials=pbft_parameters.num_nodes,
        p_successes=p_failure,
        min_success=min_unsafe_failures,
        max_success=max_unsafe_failures,
    )


def p_poisson_pbft_live(pbft_parameters: PbftParameters, p_failure: list[float]) -> float:
    min_unlive_failures = pbft_min_unlive_failures(pbft_parameters)
    max_unlive_failures = pbft_max_unlive_failures(pbft_parameters)

    return 1 - p_poisson_binom_range(
        num_trials=pbft_parameters.num_nodes,
        p_successes=p_failure,
        min_success=min_unlive_failures,
        max_success=max_unlive_failures,
    )


def p_poisson_pbft_safe_and_live(pbft_parameters: PbftParameters, p_failure: list[float]) -> float:
    min_unlive_failures = pbft_min_unlive_failures(pbft_parameters)
    max_unlive_failures = pbft_max_unlive_failures(pbft_parameters)
    min_unsafe_failures = pbft_min_unsafe_failures(pbft_parameters)
    max_unsafe_failures = pbft_max_unsafe_failures(pbft_parameters)

    min_unsafe_or_unlive_failures = min(min_unlive_failures, min_unsafe_failures)
    max_unsafe_or_unlive_failures = max(max_unlive_failures, max_unsafe_failures)

    return 1 - p_poisson_binom_range(
        num_trials=pbft_parameters.num_nodes,
        p_successes=p_failure,
        min_success=min_unsafe_or_unlive_failures,
        max_success=max_unsafe_or_unlive_failures,
    )


def p_poisson_raft_safe(raft_parameters: RaftParameters, p_failure: list[float]) -> float:
    min_unsafe_failures = raft_min_unsafe_failures(raft_parameters)
    max_unsafe_failures = raft_max_unsafe_failures(raft_parameters)

    return 1 - p_poisson_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_successes=p_failure,
        min_success=min_unsafe_failures,
        max_success=max_unsafe_failures,
    )


def p_poisson_raft_live(raft_parameters: RaftParameters, p_failure: list[float]) -> float:
    min_unlive_failures = raft_min_unlive_failures(raft_parameters)
    max_unlive_failures = raft_max_unlive_failures(raft_parameters)

    return 1 - p_poisson_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_successes=p_failure,
        min_success=min_unlive_failures,
        max_success=max_unlive_failures,
    )


def p_poisson_raft_safe_and_live(raft_parameters: RaftParameters, p_failure: list[float]) -> float:
    min_unlive_failures = raft_min_unlive_failures(raft_parameters)
    max_unlive_failures = raft_max_unlive_failures(raft_parameters)
    min_unsafe_failures = raft_min_unsafe_failures(raft_parameters)
    max_unsafe_failures = raft_max_unsafe_failures(raft_parameters)

    min_unsafe_or_unlive_failures = min(min_unlive_failures, min_unsafe_failures)
    max_unsafe_or_unlive_failures = max(max_unlive_failures, max_unsafe_failures)

    return 1 - p_poisson_binom_range(
        num_trials=raft_parameters.num_nodes,
        p_successes=p_failure,
        min_success=min_unsafe_or_unlive_failures,
        max_success=max_unsafe_or_unlive_failures,
    )


def is_strict_le(t1: Sequence[...], t2: Sequence[...]):
    """
    returns true iff t1 <= t2 for all elements
    """
    for i in range(len(t1)):
        if t1[i] > t2[i]:
            return False
    return True


def clean_array(arr: list[Sequence[...]]):
    """
    removes any elements from arr which are strictly greater than or equal to another element in all components
    used to remove parameters which have bigger (worse) quorum sizes __unless__ they have smaller chance to fail

    current alg -- just d*n^2 checks
    better alg O(d*n*logn) -- make d many max heaps, initilize with one element.
    For every element, if it would be added to the top of all heaps, discard it. otherwise add it
    """
    cleaned_arr = []
    unique_arr = list(set(arr))
    for elem in unique_arr:
        is_elem_useful = True
        for other_elem in cleaned_arr:
            if elem != other_elem and is_strict_le(other_elem, elem):
                is_elem_useful = False
                break
        if is_elem_useful:
            cleaned_arr.append(elem)
    final_arr = []
    for elem in cleaned_arr:
        is_elem_useful = True
        for other_elem in cleaned_arr:
            if elem != other_elem and is_strict_le(other_elem, elem):
                is_elem_useful = False
                break
        if is_elem_useful:
            final_arr.append(elem)
    return final_arr

In [ ]:
# Calculate All quorum size selections for PBFT
prob_failure = 0.01
min_num_nodes = 7
max_num_nodes = 8
pbft_aggeragate_results = []

for num_nodes in range(min_num_nodes, max_num_nodes + 1):
    for prepare_qs, commit_qs, view_change_qs, view_sync_qs in product(
        range(0, num_nodes), repeat=4
    ):
        pbft_parameters = PbftParameters(
            num_nodes=num_nodes,
            prepare_qs=prepare_qs,
            commit_qs=commit_qs,
            view_change_qs=view_change_qs,
            view_sync_qs=view_sync_qs,
        )

        safe_and_live_prob = p_pbft_safe_and_live(pbft_parameters, prob_failure)
        safety_prob = p_pbft_safe(pbft_parameters, prob_failure)
        live_prob = p_pbft_live(pbft_parameters, prob_failure)

        pbft_aggeragate_results.append(
            (
                num_nodes,
                safety_prob,
                live_prob,
                safe_and_live_prob,
                prepare_qs,
                commit_qs,
                view_change_qs,
                view_sync_qs,
            )
        )

print(
    f"Aggregrated {len(pbft_aggeragate_results)} many results for all RSMs sizes {min_num_nodes}-{max_num_nodes}"
)

In [ ]:
# Swap safe/live prob with unsafe/unlive prob so clean_array gets rid of more results which are more unsafe/unlive
pbft_altered_aggeregate_results = [
    (x[0], 1 - x[1], 1 - x[2], 1 - x[3], x[4], x[5], x[6], x[7])
    for x in pbft_aggeragate_results
]
pbft_cleaned_results = clean_array(pbft_altered_aggeregate_results)
pbft_cleaned_results = [
    (x[0], 1 - x[1], 1 - x[2], 1 - x[3], x[4], x[5], x[6], x[7])
    for x in pbft_cleaned_results
]
pbft_cleaned_results.sort()

print(
    f"After cleaning results there are {len(pbft_cleaned_results)} many unique results"
)

In [ ]:
df = pd.DataFrame(
    pbft_cleaned_results,
    columns=[
        "network_size",
        "safe_prob",
        "live_prob",
        "safe_and_live_prob",
        "x1",
        "x2",
        "x3",
        "x4",
    ],
)

print(df.to_string(float_format='%.13f'))

In [ ]:
# Calculate All quorum size selections for RAFT
prob_failure = 0.1
min_num_nodes = 20
max_num_nodes = 20
raft_aggeragate_results = []

for num_nodes in range(min_num_nodes, max_num_nodes + 1):
    for commit_qs, leader_election_qs in product(range(0, num_nodes), repeat=2):
        raft_parameters = RaftParameters(
            num_nodes=num_nodes,
            commit_qs=commit_qs,
            leader_election_qs=leader_election_qs,
        )

        safe_and_live_prob = p_raft_safe_and_live(raft_parameters, prob_failure)
        safety_prob = p_raft_safe(raft_parameters, prob_failure)
        live_prob = p_raft_live(raft_parameters, prob_failure)
        # durable_prob = 1 - p_binom_range(num_trials=num_nodes, p_success=prob_failure, min_success=raft_parameters.commit_qs)

        if safety_prob != 0:
            raft_aggeragate_results.append(
                (
                    num_nodes,
                    safety_prob,
                    live_prob,
                    safe_and_live_prob,
                    0,#durable_prob,
                    commit_qs,
                    leader_election_qs,
                )
            )

print(
    f"Aggregrated {len(raft_aggeragate_results)} many results for all RSMs sizes {min_num_nodes}-{max_num_nodes}"
)

In [ ]:
# Swap safe/live prob with unsafe/unlive prob so clean_array gets rid of more results which are more unsafe/unlive unless they have smaller quorums
raft_altered_aggeregate_results = [
    (x[0], 1 - x[1], 1 - x[2], 1 - x[3], 1 - x[4], x[5], x[6]) for x in raft_aggeragate_results
]
raft_cleaned_results = clean_array(raft_altered_aggeregate_results)
raft_cleaned_results = [
    (x[0], 1 - x[1], 1 - x[2], (1 - x[3]), 1 - x[4], x[5], x[6]) for x in raft_cleaned_results
]
raft_cleaned_results.sort()

print(
    f"After cleaning results there are {len(raft_cleaned_results)} many unique results"
)

In [ ]:
df = pd.DataFrame(
    raft_cleaned_results,
    columns=[
        "network_size",
        "safe_prob",
        "live_prob",
        "safe_and_live_prob",
        "durable_prob",
        "x1",
        "x2",
    ],
)

print(df.to_string(float_format='%.15f'))

In [ ]:
p_bad = Decimal('1E-10')
p_lame = (1-Decimal('.9')**10)
num_trials = Decimal('20')

p_good = (1-p_bad)**num_trials
p_not_all_lame = 1 - (1-p_lame)**num_trials
print(p_good)
print(p_not_all_lame)